# Tema 7: Reconocimiento de entidades nombradas

## Ejercicio 1
NER con NLTK y SpaCy.

In [1]:
text = "Barack Obama was the 44th president of the United States"

### Apartado a
NER con NLTK (binary=False y binary=True).

In [2]:
import nltk

In [3]:
tokenized_sentence = nltk.word_tokenize(text)
print(tokenized_sentence)

print()

tagged_sentence = nltk.pos_tag(tokenized_sentence)
print(tagged_sentence)

['Barack', 'Obama', 'was', 'the', '44th', 'president', 'of', 'the', 'United', 'States']

[('Barack', 'NNP'), ('Obama', 'NNP'), ('was', 'VBD'), ('the', 'DT'), ('44th', 'JJ'), ('president', 'NN'), ('of', 'IN'), ('the', 'DT'), ('United', 'NNP'), ('States', 'NNPS')]


In [4]:
for tree in nltk.ne_chunk(tagged_sentence, binary=False):
    if isinstance(tree, nltk.Tree):
        ent_name = " ".join([token for token, _ in tree.leaves()])
        ent_type = tree.label()

        print(tree)
        print(ent_name, ent_type)
        print()

(PERSON Barack/NNP)
Barack PERSON

(PERSON Obama/NNP)
Obama PERSON

(GPE United/NNP States/NNPS)
United States GPE



In [5]:
for tree in nltk.ne_chunk(tagged_sentence, binary=True):
    if isinstance(tree, nltk.Tree):
        ent_name = " ".join([token for token, _ in tree.leaves()])
        ent_type = tree.label()

        print(tree)
        print(ent_name, ent_type)
        print()

(NE Barack/NNP Obama/NNP)
Barack Obama NE

(NE United/NNP States/NNPS)
United States NE



### Apartado b
NER con SpaCy (en_core_web_sm).

In [6]:
import spacy


nlp = spacy.load("en_core_web_sm")

In [7]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

Barack Obama PERSON
44th ORDINAL
the United States GPE


### Apartado c
NER con SpaCy Transformer (en_core_web_trf).

In [8]:
nlp = spacy.load("en_core_web_trf")

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

Barack Obama PERSON
44th ORDINAL
the United States GPE


### Apartado d
Comparar modelos con múltiples frases.

In [9]:
def get_entities_nltk(text: str, binary=False):
    tokenized_sentence = nltk.word_tokenize(text)
    tagged_sentence = nltk.pos_tag(tokenized_sentence)

    entities = []
    for tree in nltk.ne_chunk(tagged_sentence, binary=binary):
        if isinstance(tree, nltk.Tree):
            ent_name = " ".join([token for token, _ in tree.leaves()])
            ent_type = tree.label()
            entities.append((ent_name, ent_type))

    return entities

In [10]:
def get_entities_spacy(text: str, nlp):
    return [
        (ent.text, ent.label_)
        for ent in nlp(text).ents
    ]

In [11]:
sentences = [
    "Apple is looking at buying U.K. startup for $1 billion",
    "The Eiffel Tower is located in Paris, France",
    "Cristiano Ronaldo scores winner in Champions League final"
]

In [12]:
nlp_sm = spacy.load("en_core_web_sm")
nlp_trf = spacy.load("en_core_web_trf")


for sent in sentences:
    print('-----------------------------------------------------------')
    print('Sentence:', sent)
    print('-----------------------------------------------------------')
    print('NLTK (binary=False):', get_entities_nltk(sent, binary=False))
    print('NLTK (binary=True):', get_entities_nltk(sent, binary=True))
    print('SpaCy (CNN):', get_entities_spacy(sent, nlp_sm))
    print('SpaCy (Transformer):', get_entities_spacy(sent, nlp_trf))
    print('-----------------------------------------------------------')

-----------------------------------------------------------
Sentence: Apple is looking at buying U.K. startup for $1 billion
-----------------------------------------------------------
NLTK (binary=False): [('Apple', 'GPE')]
NLTK (binary=True): [('Apple', 'NE')]
SpaCy (CNN): [('Apple', 'ORG'), ('U.K.', 'GPE'), ('$1 billion', 'MONEY')]
SpaCy (Transformer): [('Apple', 'ORG'), ('U.K.', 'GPE'), ('$1 billion', 'MONEY')]
-----------------------------------------------------------
-----------------------------------------------------------
Sentence: The Eiffel Tower is located in Paris, France
-----------------------------------------------------------
NLTK (binary=False): [('Eiffel Tower', 'ORGANIZATION'), ('Paris', 'GPE'), ('France', 'GPE')]
NLTK (binary=True): [('Eiffel Tower', 'NE'), ('Paris', 'NE'), ('France', 'NE')]
SpaCy (CNN): [('The Eiffel Tower', 'FAC'), ('Paris', 'GPE'), ('France', 'GPE')]
SpaCy (Transformer): [('The Eiffel Tower', 'FAC'), ('Paris', 'GPE'), ('France', 'GPE')]
-----

### Apartado e
NER en dominio biomédico con SciSpaCy.

In [13]:
nlp = spacy.load("en_ner_jnlpba_md")

text = "The p53 protein regulates the expression of CDKN1A mRNA in HeLa cells."

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

p53 protein PROTEIN
CDKN1A mRNA RNA
HeLa cells CELL_LINE


## Ejercicio 2
Fine-tuning de NER con SpaCy.

### Apartado a
NER con modelo por defecto (es_core_news_sm) sobre texto periodístico.

In [14]:
from pprint import pprint


text = """En medio de una cuidada puesta en escena digna del fin de una guerra, y ante más de 50 cámaras de medios, algunos venidos de fuera de España para la ocasión, Carmen Cervera, el Ministro de Cultura Miquel Iceta, en representación del Gobierno del Estado español, y Borja Thyssen-Bornemisza han procedido esta mañana a la dilatada firma del acuerdo que permite la permanencia en España del Mata Mua y una parte importante de la colección de la baronesa Thyssen-Bornemisza. El cuadro más famoso de pintor postimpresionista francés Paul Gauguin ya cuelga en las salas del museo madrileño. Ha sido una larga travesía en las que se han desarrollado intensísimas negociaciones, retiradas de las partes, principios de acuerdo y retrasos desde hace casi una década. \"Ha sido necesario mucho esfuerzo de las dos partes, pero hoy al fin es una hermosa realidad\", ha declarado una satisfecha la baronesa. \"Hasta hace pocos días no estaba seguro que el cuadro estuviese aquí entre nosotros. Hoy tenemos el honor y el privilegio de disfrutar en España de esta y otras pinturas que componen una colección de las más importantes del mundo. Esta firma es un final feliz\", concluyó el ministro de Cultura. El contrato de arrendamiento asegura la permanencia del cuadro en España, junto con 320 obras pertenecientes a la colección Carmen Thyseen-Bornemisza, una cuarta parte menos de la garantía actual, durante 15 años. A cambio, la baronesa recibirá 6,5 millones de euros anuales en calidad de préstamo. Transcurrido ese tiempo, y pagado el importe total, 97,5 millones de euros, el Estado podrá optar a la compra del cuadro, descontando del precio final lo pagado. Esto puede suponer reeditar en el futuro los problemas hoy finiquitados."""

pprint(text, width=125)

('En medio de una cuidada puesta en escena digna del fin de una guerra, y ante más de 50 cámaras de medios, algunos venidos '
 'de fuera de España para la ocasión, Carmen Cervera, el Ministro de Cultura Miquel Iceta, en representación del Gobierno '
 'del Estado español, y Borja Thyssen-Bornemisza han procedido esta mañana a la dilatada firma del acuerdo que permite la '
 'permanencia en España del Mata Mua y una parte importante de la colección de la baronesa Thyssen-Bornemisza. El cuadro '
 'más famoso de pintor postimpresionista francés Paul Gauguin ya cuelga en las salas del museo madrileño. Ha sido una larga '
 'travesía en las que se han desarrollado intensísimas negociaciones, retiradas de las partes, principios de acuerdo y '
 'retrasos desde hace casi una década. "Ha sido necesario mucho esfuerzo de las dos partes, pero hoy al fin es una hermosa '
 'realidad", ha declarado una satisfecha la baronesa. "Hasta hace pocos días no estaba seguro que el cuadro estuviese aquí '
 'entr

In [15]:
import spacy


nlp = spacy.load("es_core_news_sm")

In [16]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

España PER
Carmen Cervera PER
Ministro de Cultura Miquel Iceta LOC
Gobierno del LOC
Estado español LOC
Borja Thyssen-Bornemisza PER
España del Mata Mua MISC
Thyssen-Bornemisza LOC
El cuadro más famoso de pintor postimpresionista francés MISC
Paul Gauguin PER
Ha sido una larga travesía MISC
Hoy tenemos el honor MISC
España LOC
Esta firma MISC
Cultura LOC
El contrato de arrendamiento asegura la permanencia del cuadro MISC
España LOC
Carmen Thyseen-Bornemisza PER
Transcurrido MISC
Estado podrá LOC
Esto PER


### Apartado b
Preparar dataset de entrenamiento.

In [17]:
from spacy.training import Example

In [18]:
training = [
    (
        "Acuerdo entre el Gobierno del Estado Español y el Gobierno de la República Federal de Alemania para la aplicación del Convenio de 20 de abril de 1966 sobre Seguro de Desempleo.",
        [(17, 44, "ORG"), (50, 94, "ORG"), (156, 175, "MISC")]
    ),
    (
        "Esta mañana, el primer secretario del PSC, Miquel Iceta, ha tomado posesión del cargo de ministro de Cultura y Deportes.",
        [(16, 33, "PER"), (38, 41, "ORG"), (43, 55, "PER"), (89, 119, "PER")]
    ),
    (
        "España el país que más gusta a los franceses.",
        [(0, 6, "LOC")]
    )
]

pprint(training, width=125)

[('Acuerdo entre el Gobierno del Estado Español y el Gobierno de la República Federal de Alemania para la aplicación del '
  'Convenio de 20 de abril de 1966 sobre Seguro de Desempleo.',
  [(17, 44, 'ORG'), (50, 94, 'ORG'), (156, 175, 'MISC')]),
 ('Esta mañana, el primer secretario del PSC, Miquel Iceta, ha tomado posesión del cargo de ministro de Cultura y Deportes.',
  [(16, 33, 'PER'), (38, 41, 'ORG'), (43, 55, 'PER'), (89, 119, 'PER')]),
 ('España el país que más gusta a los franceses.', [(0, 6, 'LOC')])]


In [19]:
examples = []
for raw_text, entity_offsets in training:
    example = Example.from_dict(
        nlp.make_doc(raw_text),
        {"entities": entity_offsets}
    )
    examples.append(example)

examples

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'L-MISC', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['Acuerdo', 'entre', 'el', 'Gobierno', 'del', 'Estado', 'Español', 'y', 'el', 'Gobierno', 'de', 'la', 'República', 'Federal', 'de', 'Alemania', 'para', 'la', 'aplicación', 'del', 'Convenio', 'de', '20', 'de', 'abril', 'de', '1966', 'sobre', 'Seguro', 'de', 'Desempleo', '.'], 'SPACY': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', '', '', ''

### Apartado c
Fine-tuning con etiquetas existentes.

In [20]:
import random

In [21]:
disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
disabled_pipes

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer']

In [22]:
optimizer = nlp.create_optimizer()
with nlp.disable_pipes(*disabled_pipes):
    for _ in range(25):
        random.shuffle(training)
        for example in examples:
            nlp.update([example], sgd=optimizer)

In [23]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

España LOC
Carmen Cervera PER
Ministro de Cultura Miquel Iceta PER
Gobierno del Estado español ORG
Borja Thyssen-Bornemisza PER
España del Mata Mua MISC
Thyssen-Bornemisza LOC
El cuadro más famoso de pintor postimpresionista MISC
Paul Gauguin PER
Ha sido una larga travesía MISC
Hoy tenemos el honor MISC
España LOC
Esta firma MISC
ministro de Cultura PER
El contrato de arrendamiento MISC
España LOC
Carmen Thyseen-Bornemisza PER
Transcurrido MISC
Estado podrá LOC


### Apartado d
Fine-tuning con nueva etiqueta JOB.

In [24]:
nlp = spacy.load("es_core_news_sm")

ner = nlp.get_pipe('ner')

print("Default labels", ner.labels)
ner.add_label('JOB')
print("New labels", ner.labels)

Default labels ('LOC', 'MISC', 'ORG', 'PER')
New labels ('JOB', 'LOC', 'MISC', 'ORG', 'PER')


In [25]:
training = [
    (
        "Acuerdo entre el Gobierno del Estado Español y el Gobierno de la República Federal de Alemania para la aplicación del Convenio de 20 de abril de 1966 sobre Seguro de Desempleo.",
        [(17, 44, "ORG"), (50, 94, "ORG"), (156, 175, "MISC")]
    ),
    (
        "Esta mañana, el primer secretario del PSC, Miquel Iceta, ha tomado posesión del cargo de ministro de Cultura y Deportes.",
        [(16, 33, "JOB"), (38, 41, "ORG"), (43, 55, "PER"), (89, 119, "JOB")]
    ),
    (
        "España el país que más gusta a los franceses.",
        [(0, 6, "LOC")]
    )
]

pprint(training, width=125)

[('Acuerdo entre el Gobierno del Estado Español y el Gobierno de la República Federal de Alemania para la aplicación del '
  'Convenio de 20 de abril de 1966 sobre Seguro de Desempleo.',
  [(17, 44, 'ORG'), (50, 94, 'ORG'), (156, 175, 'MISC')]),
 ('Esta mañana, el primer secretario del PSC, Miquel Iceta, ha tomado posesión del cargo de ministro de Cultura y Deportes.',
  [(16, 33, 'JOB'), (38, 41, 'ORG'), (43, 55, 'PER'), (89, 119, 'JOB')]),
 ('España el país que más gusta a los franceses.', [(0, 6, 'LOC')])]


In [26]:
examples = []
for raw_text, entity_offsets in training:
    example = Example.from_dict(
        nlp.make_doc(raw_text),
        {"entities": entity_offsets}
    )
    examples.append(example)

examples

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'L-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'L-MISC', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['Acuerdo', 'entre', 'el', 'Gobierno', 'del', 'Estado', 'Español', 'y', 'el', 'Gobierno', 'de', 'la', 'República', 'Federal', 'de', 'Alemania', 'para', 'la', 'aplicación', 'del', 'Convenio', 'de', '20', 'de', 'abril', 'de', '1966', 'sobre', 'Seguro', 'de', 'Desempleo', '.'], 'SPACY': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', '', '', ''

In [27]:
disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
disabled_pipes

optimizer = nlp.create_optimizer()
with nlp.disable_pipes(*disabled_pipes):
    for _ in range(25):
        random.shuffle(training)
        for example in examples:
            nlp.update([example], sgd=optimizer)

In [28]:
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

España LOC
Carmen Cervera PER
Ministro de Cultura Miquel Iceta JOB
Gobierno del Estado español ORG
Borja Thyssen-Bornemisza PER
España del Mata Mua MISC
Thyssen-Bornemisza LOC
El cuadro más famoso de pintor postimpresionista MISC
Paul Gauguin PER
Hoy tenemos el honor MISC
España LOC
Esta firma MISC
ministro de Cultura. JOB
España LOC
Carmen Thyseen-Bornemisza PER
